#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Adjust model script for lfi accordingly
* Create evidence file (if I don't need different evidence files, move to builmodel notebook)
* LFI

Multiple instances, 1 pst, X enzymes, one sample per instance, multiple enzymes per instance, test how many enzymes can have in one instances, how computing time depends on N p::f for 60 samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import random

from src.models import model_schemata as schema
from src.models import build_model as build
from src.visualization import visualize as viz

In [3]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

#### Training/Testing

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/p_fc.csv')
data['p_regulates'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/p_regulates.csv')
data['e_ksea']

,enzyme,sample,tc,value,prob,p_dec,p_inc
0,ABL1,AC220,10.0,inc,0.648536,0.001000,0.648536
1,ABL1,AT13148,10.0,inc,0.590795,0.001000,0.590795
2,ABL1,AZ20,10.0,inc,0.651697,0.001000,0.651697
3,ABL1,AZD1480,10.0,inc,0.569660,0.001000,0.569660
4,ABL1,AZD3759,10.0,dec,0.836416,0.836416,0.001000
...,...,...,...,...,...,...,...
361,SRC,Torin,12.0,inc,0.774683,0.001000,0.774683
362,SRC,Trametinib,12.0,inc,0.735235,0.001000,0.735235
363,SRC,U73122,12.0,inc,0.550325,0.001000,0.550325
364,SRC,Ulixertinib,12.0,inc,0.601812,0.001000,0.601812


In [5]:
data['p_fc']['phosphosite'].value_counts()

ABL1(S569)     61
ABL1(S718)     61
ABL1(T735)     61
HIPK2(Y361)    61
PTK2(S29)      61
PTK2(S722)     61
PTK2(S910)     61
PTPRG(S995)    61
SRC(S17)       61
SRC(S75)       61
PTK2(S843)     61
Name: phosphosite, dtype: int64

Splitting data into training/testing

In [6]:
# getting sample names (union of all samples in all datasets)
samples = list(set(data['e_ksea']['sample']).union(set(data['p_fc']['sample'])))
samples.sort()
# sample x% of samples randomly without replacement with seed
random.seed(612)
train = random.sample(samples, int(len(samples)*1))
print(train)
# filter data
training_data = {}
training_data['e_ksea'] = data['e_ksea'][data['e_ksea']['sample'].isin(train)].reset_index(drop=True)
training_data['p_fc'] = data['p_fc'][data['p_fc']['sample'].isin(train)].reset_index(drop=True)

['TAK715', 'TGX221', 'CHIR99021', 'CAL101', 'Ripasudil', 'KN93', 'LY2584702', 'LY2090314', 'Ku0063794', 'SP600125', 'Ribociclib', 'U73122', 'AZD5438', 'GDC0994', 'Edelfosine', 'PF3758309', 'Amuvatinib', 'AT13148', 'AZD3759', 'Go6976', 'JNJ', 'NU7441', 'GSK2334470', 'Ipatasertib', 'AZD5363', 'LY2835219', 'Torin', 'Bosutinib', 'H89', 'DNAPK', 'GDC0941', 'FRAX486', 'CX4945', 'HS173', 'Linsitinib', 'Dabrafenib', 'AZD1480', 'BX912', 'AZ20', 'TBCA', 'Selumetinib', 'GF109203X', 'AZD6482', 'AC220', 'PH797804', 'AZD8055', 'Trametinib', 'JNK', 'GSK690693', 'KD025', 'Vemurafenib', 'PF4708671', 'Tofacitinib', 'MK2206', 'GO6983', 'Dasatinib', 'AZD6738', 'PIK294', 'PD153035', 'KN62', 'Ulixertinib']


In [7]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(training_data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(training_data['p_fc'], 'phosphosite', 'sample', 'value')

In [8]:
samples = list(set(training_data['e_ksea']['sample']).union(set(training_data['p_fc']['sample'])))
enzymes = list(set(training_data['e_ksea']['enzyme']))
evidence_dict = {}
for s in samples:
    evidence_dict[s] = {}
    for e in enzymes:
        phosphosites = data['p_regulates']['phosphosite'][data['p_regulates']['protein'] == e].tolist()
        evid_generator = build.ProblogStatementGenerator(predicates['p_fc'])
        evidence_p = []
        for p in phosphosites:
            evidence_p.extend(evid_generator.generate_facts(build.EvidenceTemplate, select=[s, p]))
        evid_generator = build.ProblogStatementGenerator(predicates['e_ksea'])
        evidence_e = evid_generator.generate_facts(build.EvidenceTemplate, select=[s, e])
        evidence_dict[s][e] = evidence_p + evidence_e

In [9]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/e_model/evidence_exp1.pl', 'w') as f:
    for s in evidence_dict:
        for e in evidence_dict[s]:
            for p in evidence_dict[s][e]:
                f.write(p + '\n')
            f.write('--------------------' + '\n')

In [55]:
# write evidence_dict to text file: loop over samples and separate by line of hyphens
with open('models/ebdt_data/sub_network/e_model/evidence_exp1_training.pl', 'w') as f:
    for s in evidence_dict:
        for e in evidence_dict[s]:
            for p in evidence_dict[s][e]:
                f.write(p + '\n')
            f.write('--------------------' + '\n')

LFI

In [6]:
DIR = 'models/ebdt_data/sub_network/e_model/'
max_iter = 100
# learning from interpretation
cmd = f'problog lfi {DIR}e_model_lfi_exp1.pl {DIR}evidence_exp1.pl -O {DIR}e_model_exp1_100i.pl --logger {DIR}log_exp1-100i.txt -k ddnnf -v -n {max_iter}'
os.system(cmd)

-788.6243957247916 [0.37159504, 0.62840496, 0.35127726, 0.64872274, 0.00592176, 0.99407824, 0.46797817, 0.53202183, 0.88354225, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.00048357, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.60039329, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.11e-05, 1.0, 0.0, 1.0, 1.04e-06, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.98752786, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 7.003e-05, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.97959247, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0

0

In [4]:
DIR = 'models/ebdt_data/sub_network/p_model/'
# learning from interpretation
cmd = f'problog ground {DIR}p_model_lfi_exp5.pl -o {DIR}p_model_exp5_ground.pl --format pl'
os.system(cmd)

(True, 't(0.33,"SRC","FRAX486")::e_activity("SRC","FRAX486",dec); t(0.33,"SRC","FRAX486")::e_activity("SRC","FRAX486",inc).\n0.7961814123::e_ksea("SRC","FRAX486",dec) :- e_activity("SRC","FRAX486",dec).\n0.001::e_ksea("SRC","FRAX486",dec) :- e_activity("SRC","FRAX486",inc).\n0.1019092939::e_ksea("SRC","FRAX486",dec).\nt(0.33,"ABL1","FRAX486")::e_activity("ABL1","FRAX486",dec); t(0.33,"ABL1","FRAX486")::e_activity("ABL1","FRAX486",inc).\n0.9852877594::e_ksea("ABL1","FRAX486",dec) :- e_activity("ABL1","FRAX486",dec).\n0.001::e_ksea("ABL1","FRAX486",dec) :- e_activity("ABL1","FRAX486",inc).\n0.0073561203::e_ksea("ABL1","FRAX486",dec).\nt(0.33,"PTK2","FRAX486")::e_activity("PTK2","FRAX486",dec); t(0.33,"PTK2","FRAX486")::e_activity("PTK2","FRAX486",inc).\n0.7740504903::e_ksea("PTK2","FRAX486",dec) :- e_activity("PTK2","FRAX486",dec).\n0.001::e_ksea("PTK2","FRAX486",dec) :- e_activity("PTK2","FRAX486",inc).\n0.1129747548::e_ksea("PTK2","FRAX486",dec).\nt(0.33,"PTPRG","FRAX486")::e_activity(

256